# Notebook to pack Zhang68k data from x10cell into an AnnData (h5ad) file.  



This notebook assumes that it is ran from the `bmfm-mammal-release/mammal/examples/scrna_cell_type` directory.  

In [ ]:
# check the current directory.  Notice that the `biomed-multi-alignment` will probably be placed in a different location on your system.
!pwd

/u/matanin/git/biomed-multi-alignment/mammal/examples/scrna_cell_type/data



## Obtaining the source data:
The main data is availble online, for example in the [10xgenomics](https://www.10xgenomics.com/) cite.  The lables are based on the data in [LINK](https://www.10xgenomics.com/datasets/fresh-68-k-pbm-cs-donor-a-1-standard-1-1-0)

From this download the file `fresh_68k_pbmc_donor_a_filtered_gene_bc_matrices.tar.gz` and place it in this directory. 

In [ ]:
!ls -sh

total 119M
 16K clear_data_prep.ipynb				       512 README.md
119M fresh_68k_pbmc_donor_a_filtered_gene_bc_matrices.tar.gz


#### Unzip the file.

In [ ]:
!tar -xzvf fresh_68k_pbmc_donor_a_filtered_gene_bc_matrices.tar.gz 

filtered_matrices_mex/
filtered_matrices_mex/hg19/
filtered_matrices_mex/hg19/barcodes.tsv
filtered_matrices_mex/hg19/genes.tsv
filtered_matrices_mex/hg19/matrix.mtx


#### download the labels file from a git repository

In [ ]:
!wget https://raw.githubusercontent.com/scverse/scanpy_usage/refs/heads/master/170503_zheng17/data/zheng17_bulk_lables.txt

--2025-02-18 11:24:49--  https://raw.githubusercontent.com/scverse/scanpy_usage/refs/heads/master/170503_zheng17/data/zheng17_bulk_lables.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1348927 (1.3M) [text/plain]
Saving to: ‘zheng17_bulk_lables.txt.1’

zheng17_bulk_lables 100%[===================>]   1.29M  --.-KB/s    in 0.03s   

2025-02-18 11:24:49 (48.3 MB/s) - ‘zheng17_bulk_lables.txt.1’ saved [1348927/1348927]



  You should now have a directoy called `filtered_matrices_mex/hg19` 

In [ ]:
!ls -shR --color=never


.:
total 121M
 16K clear_data_prep.ipynb
 512 filtered_matrices_mex
119M fresh_68k_pbmc_donor_a_filtered_gene_bc_matrices.tar.gz
 512 README.md
1.8M zheng17_bulk_lables.txt

./filtered_matrices_mex:
total 512
512 hg19

./filtered_matrices_mex/hg19:
total 485M
1.2M barcodes.tsv  800K genes.tsv  484M matrix.mtx


The output should be something like

```
.:
total 121M
 16K clear_data_prep.ipynb
 512 filtered_matrices_mex
119M fresh_68k_pbmc_donor_a_filtered_gene_bc_matrices.tar.gz
 512 README.md
1.8M zheng17_bulk_lables.txt

./filtered_matrices_mex:
total 512
512 hg19

./filtered_matrices_mex/hg19:
total 485M
1.2M barcodes.tsv  800K genes.tsv  484M matrix.mtx

## Pack the data into an AnnData file 

In [ ]:
import anndata 
# from collections import Counter
import pandas as pd
# import matplotlib.pyplot as plt
# import math
import numpy as np
# from scipy.sparse import csr_matrix
from scipy.io import mmread
import scanpy as sc

#### Read the scRNA matrix from a file


In [ ]:
mmx = mmread("filtered_matrices_mex/hg19/matrix.mtx")


#### Create an AnnData object wrapping the read data

Notice that this code transposes the data to the correct direction

In [ ]:
 
anndata_object = anndata.AnnData(X=mmx.transpose().tocsr())
print(anndata_object.X.shape)

(68579, 32738)


In [ ]:
# Cell identifiers
barcodes = pd.read_csv("filtered_matrices_mex/hg19/barcodes.tsv",header=None,sep="\t")
# names of genes
genes = pd.read_csv("filtered_matrices_mex/hg19/genes.tsv",header=None,sep="\t")


In [ ]:
# cell types (this is actualy just one column)
cell_type_lables = pd.read_csv("zheng17_bulk_lables.txt",header=None,sep="\t")

In [ ]:

# use the gene names as variable names in the AnnData object
anndata_object.var_names=genes[1]

# use the cell barcodes as names for the samples
anndata_object.obs_names=barcodes[0]



In [ ]:

# use cell types as labels for the samples
anndata_object.obs['celltype']=cell_type_lables.squeeze().to_numpy()

In [ ]:
# Save result anndata object to disk
anndata_object.write_h5ad("Zhang_68k_processed.h5ad")

In [ ]:
# process the data - filter out cells with shallow reads, normelize depth and change to log scale of about 0-10 (log_2(1001)~=10)

sc.pp.filter_cells(anndata_object,min_genes=200)
sc.pp.normalize_total(anndata_object,1000.)
sc.pp.log1p(anndata_object,base=2)


/u/matanin/git/mammal-extension/.venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/u/matanin/git/mammal-extension/.venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/tmp/ipykernel_2821082/2155608250.py:4: FutureWarning: The specified parameters ('target_sum',) are no longer positional. Please specify them like `target_sum=1000.0`
  sc.pp.normalize_total(anndata_object,1000.)


In [ ]:
anndata_object.X.data.min(), anndata_object.X.max()

(0.13107747734466052, 7.542680157842554)

In [ ]:
# split range to bins - more or less 0,2,3,..10
bins=np.linspace(anndata_object.X.data.min(), anndata_object.X.max(),num=11)
bins

array([0.13107748, 0.87223775, 1.61339801, 2.35455828, 3.09571855,
       3.83687882, 4.57803909, 5.31919935, 6.06035962, 6.80151989,
       7.54268016])

In [ ]:
# convert the counts to bins
anndata_object.X.data=np.digitize(anndata_object.X.data, bins)

In [ ]:
# Save result anndata object to disk
anndata_object.write_h5ad("Zhang68k_filtered.h5ad")

In [ ]:
def convert_to_double_sorted_geneformer_sequance(anndata_object):
    # the genes are sorted by expression bin (decending) and within the bin by the gene names.
    
    return [a[1] for a in sorted(zip(-anndata_object.X.data,anndata_object.var_names.to_numpy()[anndata_object.X.indices]))]

In [ ]:
convert_to_double_sorted_geneformer_sequance(anndata_object)

KeyboardInterrupt: 

In [ ]:
anndata_object.n_obs

AttributeError: 'AnnData' object has no attribute 'keys'